Thomas Nowakowski Project 9 - Natural Language Processing - US Airline Sentiment

In [15]:
# Importing Libraries

from bs4 import BeautifulSoup # for the removal of HTML tags 

import nltk

from nltk.tokenize.toktok import ToktokTokenizer # import tokenizer for further feeding into 
                                              # steemer/leminizer since these take tokens not strings
from nltk import word_tokenize  # another tokenizer to have some hyperparameter options

from nltk.tokenize import TweetTokenizer # this is the one that worked - others did not 

import warnings
warnings.filterwarnings('ignore')

# from nltk.corpus import stopwords, wordnet
from nltk.corpus import stopwords, wordnet # import stopwords to apply on text

from nltk.stem import LancasterStemmer, WordNetLemmatizer  # different Lemminizers to try
                                                           # as hyperparameteres
nltk.download('stopwords')                                  
nltk.download('wordnet')
from nltk.stem.wordnet import WordNetLemmatizer    
    
    
# import spacy # import a non NLTK lemminzer, we will test accuracy results with stemming and lemminizer as
             # part of my hyper parameter adjustment

import unicodedata  # to remove any accented characters that are often found in US based English
                    # because of the Spanish influence
import contractions

import re # pythons special character and punctuation mark removal

import pandas as pd # pandas dataframes
import numpy as np

import contractions 


    

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Tomek\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Tomek\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [16]:
pwd


'C:\\Users\\Tomek\\Desktop\\Artificial Intelligence\\PROJECTS\\Project 9 -NLP'

In [17]:
# import the  dataset
data=pd.read_csv('tweets.csv')

In [18]:
# print data shape
data.shape

(14640, 15)

In [19]:
# data describe ?????
# data.describe()

# Understanding of data columns


In [20]:
#  drop all columns except for  "text" and " airline sentiment" 
data1=data.loc[:14640,['text','airline_sentiment']]


In [21]:
# THE OTHER WAYTO GET THE DATASET WE WANT USING DROP
#  drop all columns except for  "text" and " airline sentiment" 
# define data1 as the varable with all the excess columns dropped
# data1=data.drop(['tweet_id','airline_sentiment_confidence','negativereason','negativereason_confidence','airline','airline_sentiment_gold','name','negativereason_gold','retweet_count','tweet_coord','tweet_created','tweet_location','user_timezone'],axis=1) 

In [22]:
# Check the data shape
data1.shape

(14640, 2)

In [23]:
# print first 5 rows of data
data1.head()

,text,airline_sentiment
0,@VirginAmerica What @dhepburn said.,neutral
1,@VirginAmerica plus you've added commercials t...,positive
2,@VirginAmerica I didn't today... Must mean I n...,neutral
3,@VirginAmerica it's really aggressive to blast...,negative
4,@VirginAmerica and it's a really big bad thing...,negative


# Text Pre-processing : Data Preparation

In [24]:
# since there are a reasonable number of records, 14,640 there is no need to  limit
# the amount of data for processing, we will use the full set available


In [25]:
# A)  HTML Tag removal

# BeautifulSoup library already imported
def  strip_html (text):
     soup=BeautifulSoup(text,'html.parser')
     clean_text=soup.get_text()
     return clean_text

data1['text']=data1['text'].apply(lambda x:strip_html(x))

data1.head()



,text,airline_sentiment
0,@VirginAmerica What @dhepburn said.,neutral
1,@VirginAmerica plus you've added commercials t...,positive
2,@VirginAmerica I didn't today... Must mean I n...,neutral
3,@VirginAmerica it's really aggressive to blast...,negative
4,@VirginAmerica and it's a really big bad thing...,negative


In [26]:
# B) Tokenize - lemmatizer and Stemmer are fed tokens not strings, hence we can perform a few  more pre-processing tasks
                # as string inputs, before tokenizing later

In [27]:
# replace contractions with full words - this is important since we are checking sentiment, and need to later filter out the 
# stop words we want to retain 

def replace_contractions (text) :
    return contractions.fix(text)

data1['text']=data1['text'].apply(lambda x: replace_contractions(x))

data1.head()

,text,airline_sentiment
0,@VirginAmerica What @dhepburn said.,neutral
1,@VirginAmerica plus you have added commercials...,positive
2,@VirginAmerica I did not today... Must mean I ...,neutral
3,@VirginAmerica it is really aggressive to blas...,negative
4,@VirginAmerica and it is a really big bad thin...,negative


In [28]:
# as we can see in line 2 "didn't" has been changed to did not ... Contractions library worked

In [29]:
# C) remove the  numbers from text as it adds no value to sentiment analysis
def remove_numbers(text):
    text=re.sub(r'\d+','',text)
    return text

data1['text']=data1['text'].apply(lambda x:remove_numbers(x))

In [30]:
data1.head()

,text,airline_sentiment
0,@VirginAmerica What @dhepburn said.,neutral
1,@VirginAmerica plus you have added commercials...,positive
2,@VirginAmerica I did not today... Must mean I ...,neutral
3,@VirginAmerica it is really aggressive to blas...,negative
4,@VirginAmerica and it is a really big bad thin...,negative


In [31]:
# convert accented characters and turn them to regular English letters
# unicode library already imported above

def remove_accented(text): 
    English_text=unicodedata.normalize('NFKD', text).encode('ascii','ignore').decode('utf-8','ignore')
    # text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return English_text

data1['text']=data1['text'].apply(lambda x:remove_accented(x))


In [32]:
# D) Remove Special Characters and Punctuations
                            # while we already removed numbers specifically as instructed, we can 
                            # also remove them using this user defined function as an option)
        
def remove_special_char_punctuations ( text,remove_digits=False) :
    pattern=r'[^a-zA-Z0-9\s]' if not remove_digits else r'[^a-zA-Z\s]'
    text=re.sub(pattern,'',text)
    return text

   #the ^ makes it NOT alphanumberc, hence letters and numbers if remove_digits is true
    # else just letters  r'[^a-zA-Z\s]'  if remove_digits is false

data1['text']=data1['text'].apply(lambda x:remove_special_char_punctuations(x,remove_digits=True))

In [33]:
data1.head()
# clearly the @ symbol and other characters have been removed  as seen below by checking the data visually

,text,airline_sentiment
0,VirginAmerica What dhepburn said,neutral
1,VirginAmerica plus you have added commercials ...,positive
2,VirginAmerica I did not today Must mean I need...,neutral
3,VirginAmerica it is really aggressive to blast...,negative
4,VirginAmerica and it is a really big bad thing...,negative


In [34]:
# E) change all words  to lowecase 
    # lower and upper case have different values and hence would be treated as different words
   # there is absolutely no need for additional features for such words, solve by lowercasing all
   
def to_lowercase (text):
    text=text.lower()
    return text

data1['text']=data1['text'].apply(lambda x:to_lowercase(x))
   
    

In [35]:
data1.head()
# lowercase worked well as we can see below

,text,airline_sentiment
0,virginamerica what dhepburn said,neutral
1,virginamerica plus you have added commercials ...,positive
2,virginamerica i did not today must mean i need...,neutral
3,virginamerica it is really aggressive to blast...,negative
4,virginamerica and it is a really big bad thing...,negative


In [36]:
#B) Tokenize in preparation for the Lemmanizing

def text_tokenizer(text):
    tokenizer=ToktokTokenizer()
    text=tokenizer.tokenize(text)
    return text

data1['text']=data1['text'].apply(lambda x: text_tokenizer(x))

In [37]:
data1.head()

,text,airline_sentiment
0,"[virginamerica, what, dhepburn, said]",neutral
1,"[virginamerica, plus, you, have, added, commer...",positive
2,"[virginamerica, i, did, not, today, must, mean...",neutral
3,"[virginamerica, it, is, really, aggressive, to...",negative
4,"[virginamerica, and, it, is, a, really, big, b...",negative


In [38]:
# DEFINE STOPWORD LIST
# BECAUSE WE ARE CHECKING FOR SENTIMENT WE MUST REMOVE KEY STOPWORDS FROM LIST THAT CHANGE MEANING
# SUCH AS NOT, DOES NOT, NO ...

stopwords=stopwords.words('english')
# stopwords already specifically imported in with libriaries above

In [39]:
print(stopwords)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [40]:
# insure meaning is not lost since we are checking for sentiment and words like no or not will 
  # reverse meaning
keep_stopwords=['no','not','nor','ma'] # contractions has already removed/changed all the xx't words
                                  # hence they are not present in the list, and do not need to be 
                                  # added to this custom list

In [41]:
# customize stopwords list 
stopwords = list(set(stopwords) - set(keep_stopwords))
print (stopwords)

['very', 'both', 'his', 'too', 'themselves', 'wouldn', 'because', 'of', 'yours', 'these', 'can', 'they', 'll', "don't", 'some', 'just', 'ourselves', "it's", 'my', 'was', "needn't", "hadn't", "you'll", 'being', 'ain', 'above', 'yourself', 'between', 'yourselves', 'should', 'couldn', 'by', 't', 'from', 'own', 'she', 'into', 'why', 'now', 'in', 'than', "won't", 'other', 'where', 'were', 'until', 'a', 'having', 'then', 'all', 'there', 'which', 'be', 'd', 'he', 'herself', "doesn't", 'through', 'it', 'isn', 'i', 'while', "you're", 'did', "hasn't", "wouldn't", 'here', 'most', 'when', "shouldn't", 'theirs', 'under', 'is', 'after', "you've", 'your', 's', 're', 'ours', 'shan', 'an', "should've", 'such', 'you', 'off', 'hadn', "couldn't", 'has', 'hasn', 'once', 'up', 'aren', "haven't", 'for', "didn't", 'its', 'do', 'further', 'didn', "that'll", 'had', 'does', 'her', 'don', 'our', 'them', 'needn', "mustn't", 'during', 'more', "shan't", 'have', 'those', 'mustn', 've', 'that', 'the', 'any', 'few', 'm

In [42]:
# remove stopwords from dataset

def remove_stopwords (words):
    new_words=[]
    for word in words:
        if word not in stopwords:
            new_words.append(word)
    return new_words
    
data1['text']=data1['text'].apply(lambda x: remove_stopwords(x))

In [45]:
# F)  lemmatize
# Lemanizer is a better tool as it checks if the word exists in a dictionary, but down side is
# that its slower on than stemmer, hence the time difference would be applicable in analysing large
# data sets. Since we are using a relatively small dataset of less than 15,000 records,  lets choose 
# lemanizer over stemmer because its a more accurate tool expected to produce better results

#nltk.download('wordnet')                          # this is already on top and imported 
# from nltk.stem.wordnet import WordNetLemmatizer  # I left it here for reference



lemmatizer=WordNetLemmatizer()

def lemmatize_words (text):
    new_words=[]
    for word in text:
        new_words.append(lemmatizer.lemmatize(word, pos='v'))
    return new_words

data1['text']=data1['text'].apply(lambda x:lemmatize_words(x))



In [46]:
data1.head()

,text,airline_sentiment
0,"[virginamerica, dhepburn, say]",neutral
1,"[virginamerica, plus, add, commercials, experi...",positive
2,"[virginamerica, not, today, must, mean, need, ...",neutral
3,"[virginamerica, really, aggressive, blast, obn...",negative
4,"[virginamerica, really, big, bad, thing]",negative


In [47]:
# G) join words back to strings in preparation of Vectorization

def join_words(text):
    return ' '.join(text)

data1['text']=data1['text'].apply(lambda x:join_words(x))

In [48]:
# H) print first 5 rows of data after pre-processing
data1.head(5)

,text,airline_sentiment
0,virginamerica dhepburn say,neutral
1,virginamerica plus add commercials experience ...,positive
2,virginamerica not today must mean need take an...,neutral
3,virginamerica really aggressive blast obnoxiou...,negative
4,virginamerica really big bad thing,negative


# Vectorization


In [49]:
# 4A) Using CountVectorizer to converta text to numbers

from sklearn.feature_extraction.text import CountVectorizer  # import the needed tools

vectorizer=CountVectorizer (max_features=1000) # limit ourselves to 1000 word vocabulary in interest
                                           # of computational time (dimentionality curse management)

vocabulary=vectorizer.fit(data['text']) # create a vocabulary list of 1000 most frequent words,
                                        # with mapped uniqure numbers 

vector=vectorizer.transform(data['text']) # change the data to matrix form, and place values in matrix

vector=vector.toarray() # convert vector to an an array, a format we can feed our classifier later

In [84]:
print (vector.shape)
print (type(vector))
print()
print(vector)

(14640, 1000)
<class 'numpy.ndarray'>

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 0 0 0]]


In [50]:
print (vectorizer.vocabulary_) # visual check of mapping numbers to 1000 words - our vocabulary

{'virginamerica': 930, 'what': 956, 'said': 746, 'plus': 677, 'you': 996, 've': 925, 'to': 873, 'the': 850, 'experience': 305, 'didn': 257, 'today': 874, 'must': 592, 'mean': 556, 'need': 597, 'take': 834, 'another': 64, 'trip': 889, 'it': 472, 'really': 707, 'entertainment': 292, 'in': 461, 'your': 997, 'amp': 61, 'they': 856, 'have': 413, 'little': 520, 'and': 63, 'big': 124, 'bad': 108, 'thing': 857, 'about': 28, 'seriously': 770, 'would': 984, 'pay': 656, '30': 17, 'flight': 334, 'for': 354, 'seats': 759, 'that': 849, 'this': 860, 'only': 632, 'flying': 348, 'yes': 993, 'every': 298, 'time': 870, 'fly': 346, 'won': 972, 'go': 384, 'away': 103, 'missed': 577, 'without': 971, 'there': 854, 'https': 448, 'co': 190, 'well': 953, 'but': 145, 'now': 615, 'do': 264, 'was': 941, 'amazing': 57, 'arrived': 85, 'an': 62, 'hour': 440, 'early': 282, 're': 702, 'too': 879, 'good': 390, 'me': 555, 'did': 256, 'know': 488, 'is': 468, 'second': 760, 'cause': 165, 'of': 619, '10': 1, '24': 14, 'lt':

In [51]:
# label
labels=data['airline_sentiment']
type(labels)

pandas.core.series.Series

In [ ]:
# 4B) Do the same operation using TDI_IDF Vectorizer

In [65]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer_tfidf= TfidfVectorizer(max_features=1000) # as before  limit vocabulary to 1000 most
                                                     # frequent words

vocabulary_tfidf=vectorizer_tfidf.fit(data['text']) # create a vocabulary list of 1000 most frequent words,
                                                # with mapped uniqure numbers 
print('IDF values are :')
print (vectorizer_tfidf.idf_)

vector_tfidf=vectorizer.transform(data['text']) #change the data to matrix form
                                                  # and place values in matrix
                                           
vector_tfidf=vector_tfidf.toarray()  # convert vector to an an array
    


IDF values are :
[7.15759389 5.39862424 6.60259704 6.26084775 6.53113808 5.97646057
 7.29574423 7.06522057 6.01687011 5.82089647 6.51404365 6.98066318
 7.22428526 7.19038371 6.17274048 6.37207339 6.69976079 5.40979754
 7.25937658 7.22428526 6.24777567 6.12567297 6.24777567 7.29574423
 6.43269801 7.19038371 5.26370492 5.79579055 4.38904557 7.15759389
 6.92801945 6.30112165 6.12567297 6.17274048 6.76293969 7.37270527
 4.44954369 7.15759389 4.64616048 5.02323659 5.56770057 5.68630631
 5.63575403 7.09507353 4.60764481 5.54172508 4.74803667 7.41352726
 6.13723379 4.11460873 7.00806215 7.06522057 6.0272329  5.39862424
 5.70123196 5.90944986 4.52779588 5.99646124 6.23487226 2.60100421
 7.19038371 4.16833413 3.74039616 2.48115385 5.00433243 5.91875226
 7.06522057 7.15759389 4.52084336 7.25937658 5.85538264 5.67892621
 7.22428526 7.25937658 6.69976079 5.94719019 6.92801945 5.8379909
 6.85391147 3.64845867 7.15759389 6.69976079 6.40192635 7.12584519
 6.62128918 6.54852982 7.29574423 4.57298788 6

In [66]:
print (vector_tfidf.shape)
print (type(vector_tfidf))
print()
print(vector_tfidf[0:1]) 

(14640, 1000)
<class 'numpy.ndarray'>

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

# CLASSIFICATION - FIT  AND EVALUATE USING BOTH TYPES OF VECTORIZATION

In [55]:
# split data into train and test set in preparation of builing and testing a model

from sklearn.model_selection import train_test_split

X_train,X_test, y_train, y_test = train_test_split(vector,labels,test_size=0.3,random_state=42)


In [56]:
# classify CountVectorizer Bags of Words

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

forest=RandomForestClassifier(n_estimators=10,n_jobs=4)

forest=forest.fit(X_train,y_train)

print(forest)

print(np.mean(cross_val_score(forest,vector,labels,cv=10)))



RandomForestClassifier(n_estimators=10, n_jobs=4)
0.7118169398907103


In [57]:
# now apply the trained model on test data

result=forest.predict(X_test)
print (result)

['positive' 'negative' 'negative' ... 'negative' 'negative' 'negative']


In [71]:
Rf_score=forest.score(X_test,y_test)
print ('The accuracy for the test data is',Rf_score)

The accuracy for the test data is 0.7527322404371585


In [ ]:
# The accuracy of the model on test data from COUNTVECTORIZER is good - 75%

# FIT AND EVALUATE TFIDF VECTORIZER  DATA


In [67]:
# SPLIT THE  TDFIDF VECTORIZER DATA INTO TRAIN AND TEST
# SPLIT LIBRARY IMPORTED ABOVE  ALREADY



XX_train,XX_test,yy_train,yy_test=train_test_split(vector_tfidf,labels,test_size=0.3,random_state=42)

# used new test and train vars in case i am running again sub-sections of this worksheet
# and would like to keep values of vars unique rather than relod old vars with new data


In [75]:
# classify TDF-IDF Vectorizer

# libraries already imported above 

forest_tfidf=RandomForestClassifier(n_estimators=10,n_jobs=4)

forest_tfidf=forest_tfidf.fit(XX_train,yy_train)

print(forest_tfidf)

print(np.mean(cross_val_score(forest_tfidf,vector_tfidf,labels,cv=10)))


RandomForestClassifier(n_jobs=4)
0.7289617486338797


In [77]:
# now apply the trained model on test data

result=forest.predict(XX_test)
print (result)

['positive' 'negative' 'negative' ... 'negative' 'negative' 'negative']


In [78]:
Rf_score_tfidf=forest.score(XX_test,yy_test)
print ('The accuracy for the TFIDF test data is',Rf_score_tfidf)

The accuracy for the TFIDF test data is 0.7527322404371585


In [ ]:
# Both models have scored high above 75% in test data and do not have significantly different results 

In [ ]:
## COMMENT ON RESULTS 
# If we need still a higher result, it can be improved by increasing the depth of the forest !!

# Another hyper variable we can use is to increase the number of words in our vocabulary from 1000 to more ...

# 6A) Summarize

Analysing text, or properly called Natural Language Processing NLP, is an important part of AI, since its applications are wide. Classifying the type of text, whether a document or inquiry, will catagorize it and place progress it to the right department for processing shortening time and human input, as well as classyfying documents into better organized and more usful data banks and resources, and lastly analysis of sentiment.

This last one is very important since it carries the inherent benefit of predictive power of AI.

Given the amout of data out there, tweeter opinions, blogs, newspapers, emails, chats, web site form inputs all this can be utilizied to calculate the mood (sentiment) towards really any subject.

Assuming that "majority rules" we can ask questions about anything such as :
1) Who will win the 2020 presidential election in USA ?
2) Will we have a stock market crash ?
3) Will there be  Corona Virus vaccine ?
4) What will be the weather next week ?

The subject range does not have limits, yet we can process text, and classify opinions by calculating and assigning a clasification, weather binary such as negative or positive, agree or disagree, high medium low or hot and cold, wet or dry .. 

In the end we can base our prediction on the opinion of the public, and arrive at an opinion based on our calculations of majority rules, (or set own threshold) and provide a prediction to the questions above based on sentiment.

In order to evaluate the text, we must first pre-process it.
Hence its important to :

- get rid of any HTML tags left over from web scraping for data- they add absolutely no predictive value


- remove numbers, perhaps in most cases unless numbers themselves are not important depending on context as we assume its the words that carry valid information to evaluate

- Special characters, punctiations, accent marks that  just aid in tone of speach  have no predictive value, hence must be removed

- conversions to lowercase are important as meaning of The and the is same, yet a different value is stored for each word, hence it increases dimentionality as it would be stored as a separate word due to capital letter difference, and we actually may miss removal or search for these words as such search just by default is lowercap - it would not catch a capitalized letter word..

- removing the same word variations - particularly verbs that conjugate and carry time tense - ran,running, run, runned etc .. have really same root word, and add much dimentionality with no value to predictive power as the word really in classification means the same thing in terms of categories which is by root word

There is a quick way to address this - a stemmer that just cuts prefixes and suffixes quickly and more or less arrives at a reduced list of words, and while a few may be garbled in general it does the job well increasing speed of computation and data reduction to valid set, at much greater magnitude that some loss of proper words - his becoming he for example by tructiating 's'. This works well with large data sets...

If we prefer accuracy over time and computing power, and/or are working with a small data set, we can use the lemmenizer that will check if the trunctated word is in the dictionary, and leave only words that do make sense - non garbled set, hence at cost of processing have a better data set full of valid words...
 

- Tokenization - as part of data reduction, we want to take out the most common conjucting and meaning less for analysis putposes words like - he she a the... that do not carry  information we are analysing for. In order to reduce these words, strings must be broken to a list of separate words, and therefore tokenized and than compared against our list of stopwords (meaningwise worthless)  to be taken out. Warning some words like no or not are important as they reverese meaning, hence depending on context may have to be taken out of the  standard stopwords list


Once data is clean, well we must pass it ovet to a classifier and seems Random Forest is the recommended algorithm. But classifiers accept numbers not text, hence we must first :


- join the tokens, back into a string, because vectorizers accept strings.
- use a vectorizer to change text to numbers - actually a matrix, then converted to an array to be fed to a classifier.

- Vectorizers themselves can be limited to say 1000 most frequently occurring words in corpus as another data reduction technique, and once a vocabulary of accepted words is mapped out as features (columns ), they can be fitted as a numbers in an array - thats is what vectorizes do.

While a simple vectorizer just counts how many times a word appears in a document by itself, and TF-IDF vectorizer also add a valuation (by log) how many documents this word appears in, hence adjusts the value in terms of appearance in a document as well as across all documents- a more thorough technique.

Once vectorized and text is transalated to numbers, we can feed the arrays to a classifyer and check its predictive power using the usual train/test data method.


In my particular case, the results of both vectorizers were very simular, likely due to a small relatively dataset used, hence 
results produced were on a small sample. Also likely, because the data was well cleaned up- there is no reason why the predictive power should differ, on a good set of clean data using the same classification algorithm Random Forest/.



A shortcut we could use in predictive sentiment analysis of special built libraries for example  VADER or Text Blob - which have alredy pre built list of classified words (positive/ negatve groups).

These models can also indicate strength of sentiment,  an added feature not present in standard classifiers. Also, in case we lack lables, these programs have already pre made categories  in which they can label our prediction. Lastly, a nice feature here is a measure of subjectivity - level of factual information applied versus opinions. 












a classification









